# Preprocessing

### Cropping Video

The video I downloaded has the footage on the right half, so I'm cropping the left half out.

In [3]:
import cv2
def crop_right_half(input_filename, input_folder="data/Raw Videos", output_folder="data/Final Videos"):
    """
    Crops the right half of the video from the specified input folder
    and saves the result to the specified output folder.
    
    Args:
        input_filename (str): The name of the input video file.
        input_folder (str): The folder containing raw videos.
        output_folder (str): The folder to save cropped videos.
    """
    import os
    import cv2

    input_path = os.path.join(input_folder, input_filename)
    output_path = os.path.join(output_folder, input_filename)

    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    full_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    half_width = full_width // 2

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (half_width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        right_half = frame[:, full_width//2:]
        out.write(right_half)

    cap.release()
    out.release()
    print(f"Cropped video saved to {output_path}")

In [4]:
crop_right_half("adl-26-cam0.mp4")

Cropped video saved to data/Final Videos/adl-26-cam0.mp4


# Fall Detection

### YOLOv8-pose

Detecting fall based on drop in center of gravity

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
import math

# Load YOLOv8 pose model
model = YOLO("yolov8n-pose.pt")

In [ ]:

def run_fall_detection(input_video_path):
    """
    Runs fall detection on a given video and writes the annotated output to the 'Outputs' folder.

    Args:
        input_video_path (str): Path to the input video file.
        output_video_name (str): (Optional) Name of the output video file. Defaults to 'fall_detection_output.mp4'.

    Returns:
        str: Path to the saved annotated video.
    """
    import os

    # Ensure Outputs directory exists
    os.makedirs("Outputs", exist_ok=True)
    input_video_filename = os.path.basename(input_video_path)
    output_path = os.path.join("Outputs", input_video_filename)

    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fps = cap.get(cv2.CAP_PROP_FPS) or 20
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(
        output_path,
        fourcc,
        fps,
        (width, height)
    )

    def calculate_angle(p1, p2):
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        angle = abs(math.degrees(math.atan2(dy, dx)))
        return angle

    while cap.isOpened():
        ret, frame = cap.read()
        frame_height, frame_width = frame.shape[:2]
        if not ret:
            break

        results = model(frame)

        for result in results:
            if result.keypoints is None:
                continue

            keypoints = result.keypoints.xy.cpu().numpy()
            boxes = result.boxes.xyxy.cpu().numpy()

            for i, kps in enumerate(keypoints):
                box = boxes[i]
                x1, y1, x2, y2 = map(int, box)
                left_shoulder = kps[5]
                right_shoulder = kps[6]
                left_hip = kps[11]
                right_hip = kps[12]
                nose = kps[0]
                left_eye = kps[1]
                right_eye = kps[2]

                head_y = np.mean([nose[1], left_eye[1], right_eye[1]])

                # Compute torso center
                shoulder_center = (left_shoulder + right_shoulder) / 2
                hip_center = (left_hip + right_hip) / 2

                # Draw bounding box & person
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.circle(frame, tuple(shoulder_center.astype(int)), 5, (255,0,0), -1)
                cv2.circle(frame, tuple(hip_center.astype(int)), 5, (0,255,0), -1)

                # Compute vertical drop
                vertical_drop = hip_center[1] - shoulder_center[1]
                torso_length = np.linalg.norm(shoulder_center - hip_center)
                if torso_length > 0:
                    drop_ratio = vertical_drop / torso_length
                    # Empirical fall detection: drop_ratio > 0.7 (adjust as needed)

                    # TORSO ANGLE 
                    dx = shoulder_center[0] - hip_center[0]
                    dy = shoulder_center[1] - hip_center[1]

                    angle = np.degrees(np.arctan2(dx, dy))

                    is_horizontal = abs(angle) > 10

                    # BOX RATIO
                    box_ratio = (x2 - x1) / (y2 - y1)

                    is_wide = box_ratio > 1.2

                    # HEAD LOW
                    head_low = head_y > frame_height * 0.7

                    is_fall = is_horizontal and is_wide and head_low

                    label = "FALL [{}°]".format(angle) if is_fall else "No Fall [{}°]".format(angle)
                    color = (0,0,255) if is_fall else (0,255,0)
                    cv2.putText(
                        frame,
                        label,
                        (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, color, 2
                    )
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        out.write(frame)

    cap.release()
    out.release()
    print(f"Fall detection video saved to {output_path}")
    return output_path

In [10]:
run_fall_detection("data/Final Videos/fall-01-cam0-right-half.mp4")


0: 480x640 1 person, 40.8ms
Speed: 1.2ms preprocess, 40.8ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 33.2ms
Speed: 1.0ms preprocess, 33.2ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 35.9ms
Speed: 0.8ms preprocess, 35.9ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 41.0ms
Speed: 1.1ms preprocess, 41.0ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 37.6ms
Speed: 1.3ms preprocess, 37.6ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 32.5ms
Speed: 0.8ms preprocess, 32.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 31.6ms
Speed: 0.9ms preprocess, 31.6ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 30.3ms
Speed: 0.8ms preprocess, 30.3ms inference, 0.3ms postprocess per image at shape (1, 3, 48

'Outputs/fall-01-cam0-right-half.mp4'